In [1]:
import os
import sys

import pandas as pd
import numpy as np

import networkx as nx

from basicmemnet import memnet

from sklearn.metrics import confusion_matrix

In [2]:
md_train = memnet.DSL()
md_test = memnet.DSL()

md_train.import_gml("train_graph.gml")
md_test.import_gml("test_graph.gml")

test = md_test.get_graph()
train = md_train.get_graph()

In [3]:
parent_actions = ['task_1_k_cooking', 'task_2_k_cooking_with_bowls', 'task_3_k_pouring', 'task_4_k_wiping',
                  'task_5_k_cereals', 'task_6_w_hard_drive', 'task_7_w_free_hard_drive', 'task_8_w_hammering', 
                  'task_9_w_sawing']

### Create Dictionary for Rule-based Baseline

In [105]:
dict = {}
for u, v, data in train.edges(data=True):
    relation = data.get('link_type')

    if relation == 'has_element':
        head = train.nodes[u].get('utterances')[0] # head is parent action (e.g. cooking, cereals...)
        tail = train.nodes[v].get('utterances')[0] # tail is sub-action (e.g. approach, lift...)

        if tail not in dict.keys():
            dict[tail] = {}

        if head not in dict[tail].keys():
            dict[tail][head] = 1
        else:
            dict[tail][head] += 1

In [108]:
print(dict)

{'idle': {'task_1_k_cooking': 96, 'task_2_k_cooking_with_bowls': 96, 'task_3_k_pouring': 96, 'task_4_k_wiping': 94, 'task_5_k_cereals': 115, 'task_6_w_hard_drive': 111, 'task_7_w_free_hard_drive': 100, 'task_8_w_hammering': 163, 'task_9_w_sawing': 96}, 'approach': {'task_1_k_cooking': 192, 'task_2_k_cooking_with_bowls': 192, 'task_3_k_pouring': 106, 'task_4_k_wiping': 156, 'task_5_k_cereals': 382, 'task_6_w_hard_drive': 339, 'task_7_w_free_hard_drive': 233, 'task_8_w_hammering': 303, 'task_9_w_sawing': 129}, 'lift': {'task_1_k_cooking': 96, 'task_2_k_cooking_with_bowls': 96, 'task_3_k_pouring': 129, 'task_4_k_wiping': 100, 'task_5_k_cereals': 226, 'task_6_w_hard_drive': 140, 'task_7_w_free_hard_drive': 135, 'task_8_w_hammering': 140, 'task_9_w_sawing': 87}, 'stir': {'task_1_k_cooking': 48, 'task_2_k_cooking_with_bowls': 49}, 'place': {'task_1_k_cooking': 96, 'task_2_k_cooking_with_bowls': 96, 'task_3_k_pouring': 96, 'task_4_k_wiping': 100, 'task_5_k_cereals': 201, 'task_6_w_hard_drive'

In [7]:
baseline_dict = {}

for action, children in dict.items():
    # Find the child node with the maximum count
    max_child = max(children, key=children.get)
    baseline_dict[action] = max_child

In [109]:
print(baseline_dict)

{'idle': 'task_8_w_hammering', 'approach': 'task_5_k_cereals', 'lift': 'task_5_k_cereals', 'stir': 'task_2_k_cooking_with_bowls', 'place': 'task_5_k_cereals', 'retreat': 'task_5_k_cereals', 'hold': 'task_8_w_hammering', 'pour': 'task_5_k_cereals', 'drink': 'task_3_k_pouring', 'wipe': 'task_4_k_wiping', 'cut': 'task_5_k_cereals', 'screw': 'task_6_w_hard_drive', 'hammer': 'task_8_w_hammering', 'saw': 'task_9_w_sawing'}


### Generate Predictions by Baseline Dictionary

In [59]:
real_utterances = []  # True values
predictions = []  # Top 1 predictions
predictions_probab = []  # All predictions - so this one will be the list of dictionaries, 
                         # with parent actions as keys, number of votes for that action as values.

for node in test.nodes(data=True):
    node_id = node[0]   # String, e.g. "658190c06eccd77ab5dc84d4"
    node_data = node[1] # Dictionary with type, utterances, timestamp, etc.

    if node_data['utterances'][0] in parent_actions: # If the node is a parent action node

        real_utterance = node_data['utterances'][0] 
        predicted_utterances = [] 

        # Iterate through all the sub-actions and make predictions using baseline_dict.
        for u, v, data in test.edges(node_id, data=True):
            if data.get('link_type') == 'has_element':
                sub_action = test.nodes[v].get('utterances')[0]
                prediction = baseline_dict.get(sub_action)
                predicted_utterances.append(prediction)
                    
        # Keep top 1 prediction
        predicted_utterance = max(predicted_utterances,key=predicted_utterances.count) 
        predictions.append(predicted_utterance)
        # Keep the real value
        real_utterances.append(real_utterance)
        # Keep all the predictions with corresponding votes
        predictions_probab.append({i:predicted_utterances.count(i) for i in set(predicted_utterances)})
     

### Evaluate Baseline

In [103]:
def hits_at_one(true, pred):
    # true and pred are lists

    n = len(true)
    tp = sum([int(true[i]==pred[i]) for i in range (n)])
    return tp/n

def hits_at_k(true, pred, k):
    # true is a list, pred is a list of dictionaries, 
    # e.g. {'task_5_k_cereals': 8, 'task_8_w_hammering': 4, 'task_9_w_sawing': 1}

    n = len(true)
    hits = np.zeros(n)
    
    for p in range (1, k+1):
        # get key with p-th larest value from each dictionary in the pred list
        pred_i = [sorted(pred[i], key=pred[i].get)[-p] if p <= len(pred[i]) else 'None' for i in range (n)]
        for hit in range (n):
            if hits[hit] == 0 and pred_i[hit] == true[hit]:
                hits[hit] = 1
        
    tp = sum(hits)
    return tp/n

In [69]:
hits_at_one(real_utterances, predictions)

0.1111111111111111

In [111]:
for k in range (1, 4):
    print(f'Hits@{k}: ', hits_at_k(real_utterances, predictions_probab, k))

Hits@1:  0.1111111111111111
Hits@2:  0.23148148148148148
Hits@3:  0.7685185185185185
